# 1. 数据集说明

这是一份来自[和鲸社区](https://www.kesci.com/mw/dataset/5e023cd12823a10036af49b4/file)的倒闭企业数据集，总计 6,272 条记录，大小为 2.3 M，包含 21 个字段。


In [2]:
import pandas as pd 
data = pd.read_csv('com.csv')
data.head()

,bianh,com_name,com_addr,cat,se_cat,com_des,born_data,death_data,live_days,financing,...,death_reason,invest_name,ceo_name,ceo_des,ceo_per_des,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,1,麦子金服,上海,金融,借贷,麦子金服是一家移动互联网金融服务集团，为个人和企业提供包括财富管理、股权投资、投融资咨询等综...,2015-03-01,2019-11-25,1730,B轮,...,政策监管 法律法规风险,海通开元&中缔资本,黄大容,董事长,黄大容，80后，经济学硕士，精通中英德三国语言。连续创业者，23岁创立第一家公司，25岁涉足...,NaN,NaN,NaN,NaN,NaN
1,1,拓道金服,浙江,金融,借贷,拓道金服是一家专注于汽车抵押贷款这一细分市场的P2P互联网金融公司。拓道金服通过互联网民间的...,2013-11-01,2019-11-24,2214,A轮,...,政策监管 法律法规风险,蓝山中国资本&帮实资本&宏桥资本,张罗军,创始人,杭州拓道科技有限公司执行董事兼总经理，重庆邮电学院计算机专业，创立杭州仁润科技有限公司，具有...,NaN,NaN,NaN,NaN,NaN
2,1,云柏科技,广东,医疗健康,医疗器械及硬件,云柏科技是一家智能健康检测腕表研发商，专注于智能可穿戴设备领域。主要产品包括可穿戴式多功能云...,2014-09-01,2019-11-01,1887,尚未获投,...,行业竞争,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,迷你生鲜,福建,电子商务,生鲜食品,迷你生鲜是一家会员制水果生鲜电商网站，平台产品由基地直供，采购深入全国及东南亚各地区，减少中...,2017-11-01,2019-10-29,727,尚未获投,...,现金流断裂 行业竞争,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,一生健康,北京,医疗健康,寻医诊疗,微爱康一款基于移动互联网的癌症患者互助型垂直社区产品，通过在线社会化问答和类轻博客交流模式，...,2015-01-01,2019-10-17,1750,尚未获投,...,政策监管 法律法规风险,NaN,张耀斌,CEO,张耀斌，为一生（北京）健康科技有限公司的创始人。,NaN,NaN,NaN,NaN,NaN


# 2、数据分析可视化

## 2.1 死亡公司的地区分布

In [3]:
from pyecharts import options as opts
from pyecharts.charts import Map

data['com_addr'] = data['com_addr'].apply(lambda x: x.strip())
s = data.groupby('com_addr').size()

c = (
Map()
    .add("死亡企业数量", [*s.items()], "china")
    .set_global_opts(
        title_opts=opts.TitleOpts(title="地区分布"),
        visualmap_opts=opts.VisualMapOpts(max_=200),
    )
)
c.render_notebook()

## 2.2 行业排行TOP10

In [4]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker

s = data.groupby('cat').size().sort_values(ascending=False)[:10].to_dict()

c = (
    Bar()
    .add_xaxis(list(s.keys()))
    .add_yaxis("死亡企业数量", list(s.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title="行业排行TOP10"))
)
c.render_notebook()

## 2.3 细分领域TOP20

In [5]:
s = data.groupby('se_cat').size().sort_values(ascending=False)[:20].sort_values(ascending=True).to_dict()

c = (
    Bar()
    .add_xaxis(list(s.keys()))
    .add_yaxis("死亡企业数量", list(s.values()))
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(title_opts=opts.TitleOpts(title="细分领域TOP20"))
)
c.render_notebook()

## 2.4 年份分布

In [6]:
data['born_year'] = data['born_data'].apply(lambda x: x[:4])
data['death_year'] = data['death_data'].apply(lambda x: x[:4])
s1 = data.groupby('born_year').size()
s2 = data.groupby('death_year').size()
s1 = pd.DataFrame({'year': s1.index, 'born': s1.values})
s2 = pd.DataFrame({'year': s2.index, 'death': s2.values})
s = pd.merge(s1,s2, on='year', suffixes=['born', 'death'])
s = s[s['year'] > '2008']

c = (
    Bar()
    .add_xaxis( s['year'].to_list())
    .add_yaxis("新生企业数量", s['born'].to_list())
    .add_yaxis("死亡企业数量", s['death'].to_list())
    .set_global_opts(title_opts=opts.TitleOpts(title="年份分布"))
)
c.render_notebook()


## 2.5 企业存活时长

In [7]:
def live_year(x):
    if x < 365:
        return '不到1年'
    if x < 365 * 2:
        return '1-2年'
    if x < 365 * 3:
        return '2-3年'
    if x < 365 * 4:
        return '3-4年'
    if x < 365 * 5:
        return '4-5年'
    if x < 365 * 10:
        return '5-10年'
    return '10年以上'

s = data.groupby(data['live_days'].apply(lambda x: live_year(x))).size()

from pyecharts import options as opts
from pyecharts.charts import Pie

c = (
    Pie()
    .add("", [*s.items()])
    .set_global_opts(title_opts=opts.TitleOpts(title="企业存活时长"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
c.render_notebook()

## 2.6 投资人词云

In [8]:
from pyecharts import options as opts
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType

invest = {}
for row in data['invest_name'].values:
    if not pd.isnull(row):
        for name in row.split('&'):
            invest[name] = invest.get(name, 0) + 1
               
c = (
    WordCloud()
    .add("", [*invest.items()], word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="投资人词云"))
)
c.render_notebook()

 ## 2.7 倒闭原因词云

In [9]:
death_reason = {}
for row in data['death_reason'].values:
    if not pd.isnull(row):
        for name in row.split(' '):
            death_reason[name] = death_reason.get(name, 0) + 1
               
c = (
    WordCloud()
    .add("", [*death_reason.items()], word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="倒闭原因词云"))
)
c.render_notebook()

## 2.8 ceo描述词云

In [12]:
import jieba
ceo_per_des = {}
for row in data['ceo_per_des'].values:
    if not pd.isnull(row):
        result = jieba.lcut(row)
        for name in result:
            if len(name) == 1:
                break
            ceo_per_des[name] = ceo_per_des.get(name, 0) + 1

c = (
    WordCloud()
    .add("", [*ceo_per_des.items()], word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="ceo描述词云"))
)
c.render_notebook()